<a href="https://colab.research.google.com/github/Abhi10699/google-colab-playground/blob/main/hf_gpt_2_fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install accelerate
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00


In [ ]:
!unzip dad_jokes.zip

Archive:  dad_jokes.zip
  inflating: reddit_dadjokes.csv     


In [ ]:
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd

from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
df = pd.read_csv("/content/reddit_dadjokes.csv")
df = df[['joke']]
print(df.shape)
df = df.iloc[:10_000]
print(df.shape)

(216328, 1)
(10000, 1)


In [ ]:
class JokesDataset(Dataset):
  def __init__(self, data_path, tokenizer, dataset_size = 10_000):
    # read csv file
    df = pd.read_csv(data_path)

    # extract only necessary columns
    df = df[['joke']].iloc[dataset_size: dataset_size]
    df['joke_input'] = df['joke'].apply(lambda x: "JOKE:<bos>"+x+"<eos>")

    #update tokenizer

    tokenizer.add_special_tokens({"pad_token": "<pad>", "bos_token": "<bos>", "eos_token": "<eos>"})
    tokenizer.add_tokens(["JOKE:"])

    # text_arr
    text_tokenized = tokenizer(
        list(df['joke_input'].values),
        padding=True,
        truncation=True,
        max_length=40,
        return_tensors="pt"
    )

    self.input_ids = text_tokenized['input_ids']
    self.attention_masks = text_tokenized['attention_mask']
    pass

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, index):
    return (self.input_ids[index], self.attention_masks[index])

In [ ]:
model_id = "abhirajeshbhai/gpt2_dad_jokes"

tokenizer = GPT2Tokenizer.from_pretrained(model_id)
model = GPT2LMHeadModel.from_pretrained(model_id)

In [ ]:
dataset = JokesDataset("/content/dad_jokes.zip", tokenizer, dataset_size=5000)
dataset = DataLoader(dataset, batch_size=64)
model.resize_token_embeddings(len(tokenizer))

Embedding(50261, 768)

In [ ]:
EPOCHS=10
LEARNING_RATE = 5e-3
CKPT_PATH = "depressed_gpt2.ckpt"


optimizer = optim.AdamW(model.parameters(),lr=LEARNING_RATE)

model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50261, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50261, bias=False)
)

In [ ]:
def joke_infer(model, joke_seed):
  model.eval()
  joke_string = "JOKE:<bos>" if not joke_seed else f"JOKE:{joke_seed}"
  joke_string = tokenizer(joke_string, return_tensors="pt")

  input_ids = joke_string['input_ids'].to(device)
  attention_mask = joke_string['attention_mask'].to(device)

  generated_op = model.generate(input_ids, attention_mask=attention_mask)
  output = tokenizer.decode(generated_op[0])
  return output


In [ ]:
joke_infer(model)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'JOKE: <bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos>'

In [ ]:
for epoch in range(EPOCHS):
  print(f"EPOCH {epoch}/{EPOCHS}", end="\t")
  model.train()
  for input_ids, attention_masks in tqdm(dataset):
    input_ids = input_ids.to(device)
    attention_masks = attention_masks.to(device)

    optimizer.zero_grad()
    model_op = model(input_ids, attention_mask=attention_masks, labels=input_ids)
    loss = model_op.loss

    loss.backward()
    optimizer.step()
    torch.save(model.state_dict(), CKPT_PATH)

  infered_joke = joke_infer(model)
  print(infered_joke)

EPOCH 0/10	

100%|██████████| 79/79 [05:52<00:00,  4.46s/it]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


JOKE: <bos>What do a a a a a a the the the the the the the the the the
EPOCH 1/10	

100%|██████████| 79/79 [05:39<00:00,  4.30s/it]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


JOKE: <bos>What do you the the the the the the the the the the the the the the the
EPOCH 2/10	

...

JOKE: <bos>What do you call a snowman who was a hole? A other-t-t
EPOCH 8/10	

100%|██████████| 79/79 [05:37<00:00,  4.27s/it]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


JOKE: <bos>What do you call a snowman to be a common? Auld prints.<eos><pad>
EPOCH 9/10	

100%|██████████| 79/79 [05:22<00:00,  4.08s/it]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


JOKE: <bos>What do you call a cow of the road? A javascript.<eos><pad><pad><pad><pad>


In [ ]:
model.push_to_hub("gpt2_dad_jokes")

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/abhirajeshbhai/gpt2_dad_jokes/commit/74abcee2761b2adc8b230bc3d65f27e129bd421a', commit_message='Upload model', commit_description='', oid='74abcee2761b2adc8b230bc3d65f27e129bd421a', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
tokenizer.push_to_hub("gpt2_dad_jokes")

CommitInfo(commit_url='https://huggingface.co/abhirajeshbhai/gpt2_dad_jokes/commit/32b38bc32d6b721c3fb6448ce93e7e7adf039c5a', commit_message='Upload tokenizer', commit_description='', oid='32b38bc32d6b721c3fb6448ce93e7e7adf039c5a', pr_url=None, pr_revision=None, pr_num=None)

## Inference

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
model_id = "abhirajeshbhai/gpt2_dad_jokes"
model = AutoModelForCausalLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
joke_infer(model, "why do people")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


"JOKE:why do people eat the road? Because the other cow's the other cow's the other deer"